In [5]:
import pandas as pd
import duckdb
import psycopg2

In [6]:
with open('db_password.txt', 'r') as file:
    db_port = file.readline().strip()
    db_password = file.readline().strip()
    db_name = file.readline().strip()

psql_conn = psycopg2.connect(f"port = '{db_port}' dbname = '{db_name}' user = 'postgres' host = 'localhost' password = '{db_password}'")

table_names = ['item', 'lost_item', 'found_item', 'users', 'locations', 'locates', 'stores', 'returns_or_claims', 'category', 'belongs', 'posts', 'members', 'comments', 'message', 'feedback', 'notifies', 'reward'] 
con = duckdb.connect()

for table_name in table_names:
    query_str = "SELECT * FROM " + table_name
    df = pd.read_sql_query(query_str, psql_conn)
    con.register(table_name, df)

psql_conn.close()

/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_31038/3564826330.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_31038/3564826330.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/folders/y6/fdlf2f113hdd9mwtf7h1m_2r0000gn/T/ipykernel_31038/3564826330.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_str, psql_conn)
/var/fo

In [7]:
# function that sees SQL command AS input and output table from database
def query(query):
    result = con.execute(query).fetchall()
    column_names = [desc[0] for desc in con.description]
    return column_names, result

In [11]:
test_id = 'UB05836965'
columns, data = query(f'''
        SELECT 
        (SELECT username FROM USERS WHERE userid = 
            (CASE 
                WHEN m.senderid = '{test_id}' THEN m.receiverid
                ELSE m.senderid 
            END)
        ) AS the_username,
        MAX(m.mgtime) AS last_message_time
        FROM MESSAGE AS m
        WHERE m.senderid = '{test_id}' OR m.receiverid = '{test_id}'
        GROUP BY the_username
        ORDER BY last_message_time DESC;
          ''')
bad_sql = pd.DataFrame(data, columns=columns)
bad_sql

# cost=70014.21, actual_time=347.315


,the_username,last_message_time
0,上官育欣,2024-11-30 21:31:56
1,魏峰瑩,2024-11-30 21:23:23
2,毛修如,2024-11-30 05:02:39
3,古惠如,2024-11-29 20:09:55
4,邵國陽,2024-11-29 17:59:16
...,...,...
1258,曾豪傑,2023-02-05 12:38:47
1259,朱宗莉,2023-02-04 16:31:18
1260,田艾如,2023-02-04 14:36:56
1261,劉廷凌,2023-02-04 07:03:11


In [12]:
test_id = 'UB05836965'
columns, data = query(f'''
    SELECT 
        CASE 
            WHEN m.senderid = '{test_id}' THEN u_r.username
            ELSE u_s.username
        END AS the_username,
        MAX(m.mgtime) AS last_message_time
    FROM MESSAGE AS m
    JOIN USERS AS u_s ON m.senderid = u_s.userid
    JOIN USERS AS u_r ON m.receiverid = u_r.userid
    WHERE m.senderid = '{test_id}' OR m.receiverid = '{test_id}'
    GROUP BY the_username
    ORDER BY last_message_time DESC;
          ''')
good_sql = pd.DataFrame(data, columns=columns)
good_sql

# cost=42742.06, actual_time=340.212

,the_username,last_message_time
0,上官育欣,2024-11-30 21:31:56
1,魏峰瑩,2024-11-30 21:23:23
2,毛修如,2024-11-30 05:02:39
3,古惠如,2024-11-29 20:09:55
4,邵國陽,2024-11-29 17:59:16
...,...,...
1258,曾豪傑,2023-02-05 12:38:47
1259,朱宗莉,2023-02-04 16:31:18
1260,田艾如,2023-02-04 14:36:56
1261,劉廷凌,2023-02-04 07:03:11


In [ ]:
'''
CREATE INDEX idx_message_sender
ON MESSAGE (senderid);
CREATE INDEX idx_message_receiver
ON MESSAGE (receiverid);
'''

In [ ]:
test_id = 'UB05836965'
columns, data = query(f'''
    SELECT 
        CASE 
            WHEN m.senderid = '{test_id}' THEN u_r.username
            ELSE u_s.username
        END AS the_username,
        MAX(m.mgtime) AS last_message_time
    FROM MESSAGE AS m
    JOIN USERS AS u_s ON m.senderid = u_s.userid
    JOIN USERS AS u_r ON m.receiverid = u_r.userid
    WHERE m.senderid = '{test_id}' OR m.receiverid = '{test_id}'
    GROUP BY the_username
    ORDER BY last_message_time DESC;
          ''')
good_sql_with_index = pd.DataFrame(data, columns=columns)
good_sql_with_index

# cost=5665.70, actual_time=7.487

In [ ]:
'''
DROP INDEX IF EXISTS idx_message_sender;
DROP INDEX IF EXISTS idx_message_receiver;
'''